In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

# Chrome 옵션 설정
chrome_options = Options()
chrome_options.add_argument('--headless')  # 헤드리스 모드
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-gpu')  # GPU 비활성화 (필요 시)
chrome_options.add_argument('--window-size=1920,1080')  # 창 크기 설정

# Set up the webdriver (ensure the appropriate driver is installed, e.g., chromedriver)
service = webdriver.chrome.service.Service('/media/jys/ssd/workspace/008_PseudoLab/001_Stockelper/CLOVA_Studio_Stock-helper/llm-server/report_crawler/chromedriver-linux64/chromedriver')
driver = webdriver.Chrome(service=service, options=chrome_options)

try:
    # 웹페이지로 이동
    driver.get("https://seibro.or.kr/websquare/control.jsp?w2xPath=/IPORTAL/user/company/BIP_CNTS01019V.xml&menuNo=16#")
    
    # 페이지 로딩 대기
    time.sleep(5)
    
    # 테이블 요소 찾기 (실제 XPath로 변경 필요)
    table = driver.find_element(By.XPATH, '//*[@id="grid1_body_tbody"]"]')  # 실제 테이블 XPath로 변경
    
    # 헤더 추출
    headers = [header.text for header in table.find_elements(By.XPATH, './/thead//th')]
    
    # 행 추출
    rows = []
    for row in table.find_elements(By.XPATH, './/tbody//tr'):
        rows.append([cell.text for cell in row.find_elements(By.XPATH, './/td')])
    
    # 데이터프레임 생성
    df = pd.DataFrame(rows, columns=headers)
    
    # CSV로 저장
    df.to_csv('seibro_data.csv', index=False, encoding='utf-8-sig')
    
except Exception as e:
    print(f"오류 발생: {e}")
finally:
    # WebDriver 종료
    driver.quit()


오류 발생: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="YOUR_TABLE_XPATH"]"}
  (Session info: chrome=129.0.6668.70); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
#0 0x61ddbe9ed10a <unknown>
#1 0x61ddbe6d35e0 <unknown>
#2 0x61ddbe722be8 <unknown>
#3 0x61ddbe722e81 <unknown>
#4 0x61ddbe7698c4 <unknown>
#5 0x61ddbe747b4d <unknown>
#6 0x61ddbe766d7d <unknown>
#7 0x61ddbe7478c3 <unknown>
#8 0x61ddbe7156b3 <unknown>
#9 0x61ddbe71668e <unknown>
#10 0x61ddbe9b7b0b <unknown>
#11 0x61ddbe9bba91 <unknown>
#12 0x61ddbe9a4305 <unknown>
#13 0x61ddbe9bc612 <unknown>
#14 0x61ddbe98946f <unknown>
#15 0x61ddbe9dc008 <unknown>
#16 0x61ddbe9dc1d3 <unknown>
#17 0x61ddbe9ebf5c <unknown>
#18 0x7267c0494ac3 <unknown>



In [7]:
import requests
from xml.etree import ElementTree as ET

# Define the URL
url = "https://seibro.or.kr/websquare/engine/proworks/callServletService.jsp"

# Define the headers
headers = {
    "Accept": "application/xml",
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Cache-Control": "no-cache",
    "Connection": "keep-alive",
    "Content-Type": 'application/xml; charset="UTF-8"',
    "Cookie": (
        "WMONID=sXw_YYfSIuP; "
        "lastAccess=1727248203164; "
        "JSESSIONID=wHAoFxqEQB6v1oEPRvLCPcVhnoIMvTseyz7jY_Sb8nHkcpNrJET1!-391580475; "
        "globalDebug=false"
    ),
    "Host": "seibro.or.kr",
    "Origin": "https://seibro.or.kr",
    "Pragma": "no-cache",
    "Referer": "https://seibro.or.kr/websquare/control.jsp?w2xPath=/IPORTAL/user/company/BIP_CNTS01019V.xml&menuNo=16",
    "Sec-Fetch-Dest": "empty",
    "Sec-Fetch-Mode": "cors",
    "Sec-Fetch-Site": "same-origin",
    "User-Agent": (
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/128.0.0.0 Safari/537.36"
    ),
    "sec-ch-ua": '"Chromium";v="128", "Not;A=Brand";v="24", "Google Chrome";v="128"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"macOS"',
    "submissionid": "submission_entrAnalysisSummaryReportPList"
}

# Define the XML payload
xml_payload = """<?xml version="1.0" encoding="UTF-8"?>
<!-- Replace this comment with your actual XML request body -->
<YourRequest>
    <!-- Example elements -->
    <Element1>Value1</Element1>
    <Element2>Value2</Element2>
    <!-- Add more elements as needed -->
</YourRequest>
"""

# Initialize a session to manage cookies and persistent parameters
session = requests.Session()

# Optionally, update the session headers if needed
session.headers.update(headers)

try:
    # Send the POST request
    response = session.post(url, data=xml_payload)

    # Check if the request was successful
    if response.status_code == 200:
        print("Request was successful.")

        # Parse the XML response
        try:
            root = ET.fromstring(response.content)
            
            # Example: Print the root tag
            print(f"Root tag: {root.tag}")

            # Example: Iterate through child elements
            for child in root:
                print(f"{child.tag}: {child.text}")

            # TODO: Add more parsing logic based on the structure of your XML
        
        except ET.ParseError as e:
            print("Failed to parse XML response:", e)
    else:
        print(f"Request failed with status code {response.status_code}")
        print("Response content:", response.text)

except requests.exceptions.RequestException as e:
    print("An error occurred while making the request:", e)


Request was successful.
Root tag: WARNING
msg: None
level: None
detail: None
timestamp: None
